In [23]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geopy.geocoders import Nominatim

In [24]:
#load data
airports_raw = pd.read_csv('data/resumo_anual_2019.csv', sep=';', encoding='latin')
airports = airports_raw.loc[airports_raw['AEROPORTO DE ORIGEM (PAÍS)'] == 'BRASIL']

#Format and cast important numeric columns
cols = ['ASK','RPK','ATK','RTK','COMBUSTÍVEL (LITROS)','DISTÂNCIA VOADA (KM)','DECOLAGENS','BAGAGEM (KG)']
airports = airports.replace(',', '', regex=True)
airports.loc[:, cols] = airports.loc[:, cols].apply(pd.to_numeric)

#Group dataframe by airport
airports = airports.groupby('AEROPORTO DE ORIGEM (SIGLA)', as_index=False).agg({
                                                                'AEROPORTO DE ORIGEM (NOME)':'first',
                                                                'AEROPORTO DE ORIGEM (UF)':'first',
                                                                # 'ASK':'sum',
                                                                # 'RPK':'sum',
                                                                # 'ATK':'sum',
                                                                # 'RTK':'sum',
                                                                'COMBUSTÍVEL (LITROS)':'sum',
                                                                # 'DISTÂNCIA VOADA (KM)':'sum',
                                                                # 'DECOLAGENS':'sum',
                                                                # 'BAGAGEM (KG)':'sum',
                  
                            })

#Calculate energy fraction by kerosene consumption
airports['fraction'] = airports['COMBUSTÍVEL (LITROS)'] / airports['COMBUSTÍVEL (LITROS)'].sum()

#Insert Country column
airports['country'] = 'BR'

#Rename columns
airports = airports.rename({
                            'AEROPORTO DE ORIGEM (SIGLA)':'airport_id',
                            'AEROPORTO DE ORIGEM (NOME)':'name',
                            'AEROPORTO DE ORIGEM (UF)':'federal_state',
                            'COMBUSTÍVEL (LITROS)':'fuel_consumption_l'}, axis=1).set_index('airport_id')
airports

,name,federal_state,fuel_consumption_l,fraction,country
airport_id,,,,,
SBAE,AREALVA,SP,1077726.0,2.511638e-04,BR
SBAQ,ARARAQUARA,SP,4268.0,9.946564e-07,BR
SBAR,ARACAJU,SE,24723305.0,5.761761e-03,BR
SBAT,ALTA FLORESTA,MT,1079153.0,2.514964e-04,BR
SBAU,ARAÇATUBA,SP,978042.0,2.279325e-04,BR
...,...,...,...,...,...
SWMW,MAUÉS,AM,17193.0,4.006825e-06,BR
SWPI,PARINTINS,AM,479002.0,1.116313e-04,BR
SWRD,RONDONÓPOLIS,MT,4827.0,1.124931e-06,BR


In [25]:
#Load kerosene sale data
kerosene_sales = pd.read_csv('data/vendas-anuais-de-querosene-de-aviacao-por-municipio.csv', sep=';')
kerosene_sales.VENDAS = kerosene_sales.VENDAS.str.replace(',', '.').astype(float)
kerosene_sales = kerosene_sales.loc[kerosene_sales.ANO == 2019].groupby('UF').sum()
kerosene_sales['fraction'] = kerosene_sales.VENDAS / kerosene_sales.VENDAS.sum()

#Perform kerosene sales validation comparding sales of federal states
airports_grouped_uf = airports.groupby('federal_state').sum()
relative_deviation = (airports_grouped_uf.fraction - kerosene_sales.fraction)

#Check for states with fraction deviations larger than 5 %
relative_deviation[abs(relative_deviation) > 0.05]

/tmp/ipykernel_3319057/3805725813.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  kerosene_sales = kerosene_sales.loc[kerosene_sales.ANO == 2019].groupby('UF').sum()
/tmp/ipykernel_3319057/3805725813.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  airports_grouped_uf = airports.groupby('federal_state').sum()


federal_state
SP   -0.17845
Name: fraction, dtype: float64

In [26]:
#Load Airport data
airport_pub_locs =  pd.read_csv('data/AerodromosPublicos.csv', sep=';', skiprows=1, encoding='latin')
airport_priv_locs =  pd.read_csv('data/AerodromosPrivados.csv', sep=';', skiprows=1, encoding='latin')

airport_locs = pd.concat([airport_pub_locs, airport_priv_locs])

In [27]:
#Load Airport data
#Load Airport data
airport_pub_locs =  pd.read_csv('data/AerodromosPublicos.csv', sep=';', skiprows=1, encoding='latin')
airport_priv_locs =  pd.read_csv('data/AerodromosPrivados.csv', sep=';', skiprows=1, encoding='latin')

airport_locs = pd.concat([airport_pub_locs, airport_priv_locs])

#Geolocate Airports using dataframe or alternatively geolocator
geolocator = Nominatim(user_agent='airport_locations')

for airport_id, airport in airports.iterrows():
    try:
        airports.loc[airport_id, 'y'] = airport_locs.loc[airport_locs['Código OACI'] == airport_id, 'LATGEOPOINT'].values[0]
        airports.loc[airport_id, 'x'] = loc.airport_locs.loc[airport_locs['Código OACI'] == airport_id, 'LONGEOPOINT'].values[0]
    except:
        loc_string = airport_id + ', ' + airport.federal_state
        try:
            loc = geolocator.geocode(query=loc_string, country_codes='br')
            airports.loc[airport_id, 'y'] = loc.latitude
            airports.loc[airport_id, 'x'] = loc.longitude
        except:
            try:
                loc = geolocator.geocode(query=loc_string, country_codes='br')
                airports.loc[airport_id, 'y'] = loc.latitude
                airports.loc[airport_id, 'x'] = loc.longitude
            except:
                airports.loc[airport_id, 'y'] = 'not found'
                airports.loc[airport_id, 'x'] = 'not found'
airports.sort_values(by='fraction', ascending=False)

,name,federal_state,fuel_consumption_l,fraction,country,y,x
airport_id,,,,,,,
SBGR,GUARULHOS,SP,1.103387e+09,2.571440e-01,BR,-23.437795,-46.481321
SBBR,BRASÍLIA,DF,3.367763e+08,7.848565e-02,BR,-15.871748,-47.911722
SBSP,SÃO PAULO,SP,3.037006e+08,7.077736e-02,BR,-23.625687,-46.66123
SBKP,CAMPINAS,SP,2.846071e+08,6.632761e-02,BR,-23.006056,-47.141843
SBRF,RECIFE,PE,2.118865e+08,4.938011e-02,BR,-8.125678,-34.922718
...,...,...,...,...,...,...,...
SNUH,PIUMHI,MG,1.254000e+03,2.922444e-07,BR,not found,not found
SSOU,ARIPUANÃ,MT,7.210000e+02,1.680289e-07,BR,-10.187218,-59.45758
SNOS,PASSOS,MG,6.790000e+02,1.582408e-07,BR,not found,not found


In [28]:
#Check aggregated fraction of not locatable airports
airports.loc[airports.x == 'not found'].fraction.sum()

0.0024069545941167786

In [29]:
#Drop not locatable airports and adapt energy share
airports = airports.loc[airports.x != 'not found']
airports['fraction'] = airports.fuel_consumption_l / airports.fuel_consumption_l.sum()

#export airport data
airports.to_csv('data/airports.csv', index=True)

/tmp/ipykernel_3319057/713499870.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airports['fraction'] = airports.fuel_consumption_l / airports.fuel_consumption_l.sum()
